## Pearson R correlation

In [54]:
import numpy as np
import pandas as pd
import math as mt
import re
import sklearn as skl
from sklearn import preprocessing
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [2]:

# pass in column names for each CSV and read them using pandas. 
# Column names available in the readme file

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.user', sep='|', names=u_cols,
 encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.data', sep='\t', names=r_cols,
 encoding='latin-1')

#Reading items file:
i_cols = ['movie_id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.item', sep='|', names=i_cols,
 encoding='latin-1')

In [3]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
items.head()

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
ratings['rating'].unique()

array([3, 1, 2, 4, 5], dtype=int64)

In [7]:
ratings_base = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\ua.test', sep='\t', names=r_cols, encoding='latin-1')

In [8]:
cols = ['movie_id','movie title']
items2 = items[cols]

In [33]:
rating_items_raw = pd.merge(ratings,items2,how='left',on='movie_id')
rating_items_raw = rating_items_raw.drop(['unix_timestamp','movie_id'],axis=1)

In [31]:
total_users = rating_items_raw['user_id'].nunique()
#943

In [35]:
merge1 = rating_items_raw.merge(rating_items_raw,on='user_id')

In [37]:
merge1['rating_diff'] = merge1['rating_x'] - merge1['rating_y']

In [302]:
merge2 = merge1.groupby(['movie title_x','movie title_y']).agg({'user_id':'count','rating_x':'mean','rating_diff':'mean'}).reset_index()

In [303]:
minim = (merge2['rating_diff'].min())+0.01

In [304]:
merge2['rating_diff_norm'] = (merge2['rating_diff'] - (minim)) / (merge2['rating_diff'].max() - minim)

In [305]:
merge2['rating_diff_norm2'] = merge2['rating_diff_norm'].pow(2)

In [347]:
merge2['user_id'].quantile(np.arange(0,1,.01))

0.00     1.00
0.01     1.00
0.02     1.00
0.03     1.00
0.04     1.00
0.05     1.00
0.06     1.00
0.07     1.00
0.08     1.00
0.09     1.00
0.10     1.00
0.11     1.00
0.12     1.00
0.13     1.00
0.14     1.00
0.15     1.00
0.16     1.00
0.17     1.00
0.18     1.00
0.19     1.00
0.20     1.00
0.21     1.00
0.22     1.00
0.23     1.00
0.24     1.00
0.25     1.00
0.26     1.00
0.27     1.00
0.28     1.00
0.29     1.00
        ...  
0.70     8.00
0.71     9.00
0.72     9.00
0.73     9.00
0.74    10.00
0.75    10.00
0.76    11.00
0.77    12.00
0.78    12.00
0.79    13.00
0.80    14.00
0.81    15.00
0.82    16.00
0.83    17.00
0.84    18.00
0.85    19.00
0.86    20.00
0.87    22.00
0.88    23.00
0.89    25.00
0.90    27.00
0.91    29.00
0.92    32.00
0.93    36.00
0.94    39.00
0.95    44.00
0.96    50.00
0.97    59.00
0.98    72.00
0.99    96.57
Name: user_id, Length: 100, dtype: float64

In [307]:
#cut_off = 44

In [308]:
multi_item_support = merge2[merge2['movie title_x'] != merge2['movie title_y']]
#multi_item_support = multi_item_support[multi_item_support['user_id'] > cut_off]


In [309]:
multi_item_support['multi_support']=merge3['user_id']/total_users

C:\Users\suhaib.qazi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [310]:
multi_item_support['multi_support_upweight']=multi_item_support['multi_support']*(1-multi_item_support['rating_diff_norm'])

C:\Users\suhaib.qazi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [311]:
#count of users per movie

item_support = rating_items_raw.groupby(['movie title']).agg({'user_id':'count'}).reset_index()

In [312]:
item_support['support']=item_support['user_id']/total_users

In [313]:
total1 = multi_item_support.merge(item_support, left_on='movie title_x', right_on='movie title')

In [297]:
total1b = total1.drop(['movie title','user_id_y'], axis=1)

In [298]:
total2 = total1b.merge(item_support, left_on='movie title_y', right_on='movie title')

In [315]:
total2b = total2.drop(['movie title','user_id'], axis=1)

In [316]:
total2b['Confidence']= total2b['multi_support']/total2b['support_x']
total2b['Lift']= total2b['multi_support']/(total2b['support_x']*total2b['support_y'])
total2b['Lift2']= total2b['multi_support_upweight']/(total2b['support_x']*total2b['support_y'])


In [351]:
film = 'Star Wars (1977)'

In [352]:
cut_off = 100
x = total2b[(total2b['movie title_x']==film) & (total2b['user_id_x'] > cut_off)].sort_values(by=['Lift2'], ascending=False)
x['movie title_y'].head(10)

39489                   Casablanca (1942)
23448      Raiders of the Lost Ark (1981)
10084     Empire Strikes Back, The (1980)
89103          Wrong Trousers, The (1993)
26537    Shawshank Redemption, The (1994)
58882                      Vertigo (1958)
14082               Godfather, The (1972)
40180                 Citizen Kane (1941)
24326           Return of the Jedi (1983)
5051                  Blade Runner (1982)
Name: movie title_y, dtype: object

In [353]:
cut_off = 100
y = total2b[(total2b['movie title_x']==film) & (total2b['user_id_x'] > cut_off)].sort_values(by=['Lift'], ascending=False)
y['movie title_y'].head(10)

15157              Hunt for Red October, The (1990)
66045                         Batman Returns (1992)
68179                             Cinderella (1950)
85649                               Stargate (1994)
80213        Nightmare Before Christmas, The (1993)
85444          Star Trek: The Motion Picture (1979)
63345    Star Trek III: The Search for Spock (1984)
24326                     Return of the Jedi (1983)
48459                   Hudsucker Proxy, The (1994)
69341                    Conan the Barbarian (1981)
Name: movie title_y, dtype: object